<a target="_blank" href="https://colab.research.google.com/github/halsawadi/llms-bootcamp/blob/master/Langchain.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# An Introduction to Langchain

## Install packages

In [ ]:
! pip install openai python-dotenv langchain==0.0.137 pinecone-client tiktoken

In [1]:
import openai
import os

os.environ['OPENAI_API_KEY'] = None
PINECONE_API_KEY="394dd620-5ab8-4380-a44d-8b2a190913a6"
PINECONE_ENV="asia-southeast1-gcp-free"

In [2]:
# Run basic query with OpenAI wrapper

from langchain.llms import OpenAI
llm = OpenAI(model_name="text-davinci-003")
llm("explain large language models in one sentence")

'\n\nLarge language models are AI models that can process and understand natural language at scale, usually with millions of parameters.'

In [3]:
# import schema for chat messages and ChatOpenAI in order to query chatmodels GPT-3.5-turbo or GPT-4

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

In [4]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0.3)
messages = [
    SystemMessage(content="You are an expert data scientist"),
    HumanMessage(content="Write a Python script that trains a neural network on simulated data ")
]
response=chat(messages)

print(response.content,end='\n')

Sure, here's an example script that trains a simple neural network on simulated data using the Keras library:

```python
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

# Generate simulated data
np.random.seed(42)
X = np.random.rand(1000, 10)
y = np.random.randint(2, size=(1000, 1))

# Define the neural network architecture
model = Sequential()
model.add(Dense(32, input_dim=10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=10, batch_size=32)

# Evaluate the model on new data
X_test = np.random.rand(100, 10)
y_test = np.random.randint(2, size=(100, 1))
loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)
```

In this example, we generate 1000 samples of simulated data with 10 features and a binary label. We then define a simple neural n

In [5]:
# Import prompt and define PromptTemplate

from langchain import PromptTemplate

template = """
You are an expert data scientist with an expertise in building deep learning models. 
Explain the concept of {concept} in a couple of lines
"""

prompt = PromptTemplate(
    input_variables=["concept"],
    template=template,
)

In [6]:
# Run LLM with PromptTemplate

llm(prompt.format(concept="autoencoder"))

'\nAn autoencoder is a type of artificial neural network used to learn efficient data representations (or encodings) by learning to reconstruct its own inputs. It is composed of an encoder that maps the input data into a lower-dimensional representation, and a decoder that reconstructs the input data from the lower-dimensional representation.'

In [7]:
# Import LLMChain and define chain with language model and prompt as arguments.

from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("autoencoder"))


An autoencoder is a type of artificial neural network that is used to learn efficient representations of data (known as "encodings") by learning to reconstruct its input. It consists of two parts, an encoder which compresses the input data into a condensed representation, and a decoder which reconstructs the input data from the condensed representation.


In [8]:
# Define a second prompt 

second_prompt = PromptTemplate(
    input_variables=["ml_concept"],
    template="Turn the concept description of {ml_concept} and explain it to me like I'm five in 500 words",
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [9]:
# Define a sequential chain using the two chains above: the second chain takes the output of the first chain as input

from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)

# Run the chain specifying only the input variable for the first chain.
explanation = overall_chain.run("autoencoder")
print(explanation)



> Entering new SimpleSequentialChain chain...

Autoencoders are neural network architectures which are used for unsupervised learning. They learn to represent data in a lower dimension, usually by learning a compressed representation of the input data. This representation is then used to reconstruct the original data. Autoencoders are useful for data compression, representation learning, anomaly detection, and more.


Autoencoders are tools that help computers learn without us telling them what to do. Imagine that you have a box full of different shapes. You want to find a way to store them in a way that takes up less space but still lets you know what each one looks like. An autoencoder can help you do just that. 

An autoencoder is like a special type of machine learning tool. It takes the shapes in the box and looks at them very carefully. It then tries to figure out a way to represent the shapes in a simplified way. This is called a compressed representation. 

The autoencoder ta

In [10]:
# Import utility for splitting up texts and split up the explanation given above into document chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap  = 0,
)

texts = text_splitter.create_documents([explanation])


In [11]:
# Individual text chunks can be accessed with "page_content"

texts[0].page_content

'Autoencoders are tools that help computers learn without us telling them what to do. Imagine that'

In [12]:
# Import and instantiate OpenAI embeddings

from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model_name="ada")

In [13]:
# Turn the first text chunk into a vector with the embedding

query_result = embeddings.embed_query(texts[0].page_content)
print(query_result)

[-0.02850554505348402, 0.03459927060360271, 0.004296791731316673, 0.014671971992623415, 0.02239137138495726, 0.020458966453395365, -0.028362404499337256, -0.0001881923231788355, 0.03834138803098781, -0.026910543388880964, -0.050058517506678474, -0.028934968578569458, 0.027258172577688988, 0.03625561662342997, 0.028689583707092067, -0.020254477818734105, 0.04936326285435272, -0.018945759125069725, 0.01623629961696922, 0.0035861979633865083, 0.026562914200072944, -0.011512640841029405, 0.063186608130719, -0.013690432506713842, 0.02744221123129703, 0.019927298610979298, 0.009866517949083609, 0.03596933551513644, -0.025131503070669865, -0.030366381570617683, -0.007882989927533824, -0.01633854393429985, 0.024886118199192474, 0.03367907547291734, 0.013700656565917875, 0.006834991728491601, 0.02639932552751811, -0.014978703081970156, -0.006814543144422247, -0.018444765322910905, 0.015111619576912885, -0.013250783991101798, 0.014109631041272674, 0.01928316239202857, -0.03155240782854336, -0.00

In [14]:
# Import and initialize Pinecone client

import os
import pinecone
from langchain.vectorstores import Pinecone


pinecone.init(
    api_key=PINECONE_API_KEY,  
    environment=PINECONE_ENV
)

/Users/hassanalsawadi/projects/llms-bootcamp/.venv/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [15]:
# Upload vectors to Pinecone

index_name = "langchain-quickstart"
search = Pinecone.from_documents(texts, embeddings, index_name=index_name)

In [16]:
# Do a simple vector similarity search

query = "What is magical about an autoencoder?"
result = search.similarity_search(query)

print(result)

[Document(page_content='An autoencoder is like a special type of machine learning tool. It takes the shapes in the box and', metadata={}), Document(page_content='less space but still lets you know what each one looks like. An autoencoder can help you do just', metadata={}), Document(page_content='The autoencoder takes the shapes and makes them into a smaller number of shapes that are simpler and', metadata={}), Document(page_content='The autoencoder can then use the simplified representation to recreate the original shapes. So if', metadata={})]


In [17]:
# Import Python REPL tool and instantiate Python agent

from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI

agent_executor = create_python_agent(
    llm=OpenAI(temperature=0, max_tokens=1000),
    tool=PythonREPLTool(),
    verbose=True
)

In [18]:
# Execute the Python agent

agent_executor.run("Find the roots (zeros) if the quadratic function 3 * x**2 + 2*x -1")



> Entering new AgentExecutor chain...
 I need to solve a quadratic equation
Action: Python REPL
Action Input: import numpy as np
Observation: 
Thought: I can use the numpy function to solve the equation
Action: Python REPL
Action Input: np.roots([3,2,-1])
Observation: 
Thought: I now know the final answer
Final Answer: (-1.0, 0.3333333333333333)

> Finished chain.


'(-1.0, 0.3333333333333333)'